In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.1374518871307373
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 64.25479114122385
200000 106.40434143899874
300000 107.72289898417733
400000 108.00554908359656
500000 103.8236712483385
600000 104.28455926678643
700000 109.43124201208205
800000 104.1135272025409
900000 103.25763436523276
1000000 102.91671695999572
1100000 103.0823898404957
1200000 102.4524342277252
1300000 83.81058281626756
1400000 41.08239917100754
1500000 102.4461750100659
1600000 101.66516446897823
1700000 102.14060780224516
1800000 100.19790426274808
1900000 101.76526119865571
2000000 106.63544019537673
2100000 102.46733094450948
2200000 101.1586805320539
2300000 101.2208205134717
2400000 102.37462934471326
2500000 103.71113756186918
2600000 99.09310080208903
2700000 25.24926478238666
2800000 99.14754703136958
2900000 98.28930188599287
3000000 97.8

29400000 110.5263040933005
29500000 108.98169887622889
29600000 106.9557908608562
29700000 108.66513128252203
29800000 108.68184818006121
29900000 108.07421851187605
30000000 106.37878675508259
30100000 103.2870173124482
30200000 92.57028136135195
30300000 38.00629778883445
30400000 99.83853241250971
30500000 104.8902537549177
30600000 104.67865762551413
30700000 114.13584328147046
30800000 106.8954549642951
30900000 103.62253422761994
31000000 108.26938217350288
31100000 109.82593589388024
31200000 106.12027440023124
31300000 101.73639817172233
31400000 45.56757407684546
31500000 110.97414120469072
31600000 112.4558600714064
31700000 113.63752264059262
31800000 52.13132749486647
31900000 95.46839055695823
32000000 111.90913806736394
32100000 111.622998827256
32200000 113.01898740737698
32300000 109.00806955669019
32400000 105.6644414914364
32500000 108.77907050852816
32600000 107.57579521692834
32700000 107.1766808142652
32800000 104.34229247095574
32900000 25.347721252609798
33000000

59200000 23.646727576602025
59300000 101.2958386488265
59400000 100.64736172602342
59500000 102.17914082961893
59600000 103.69530904454427
59700000 106.37399381002439
59800000 105.23857145213346
59900000 106.23199491177236
60000000 105.99284064640698
60100000 46.27200057071001
60200000 112.65424312063627
60300000 110.17158317781897
60400000 111.8478888653778
60500000 109.264192924052
60600000 110.09157407561494
60700000 109.41199018270888
60800000 108.54341883670456
60900000 68.49129525754263
61000000 83.38396448686407
61100000 105.48613506233976
61200000 103.85266229746281
61300000 103.14017935500674
61400000 109.82346452354986
61500000 101.2577929594588
61600000 96.69308297425674
61700000 96.70154045347361
61800000 35.67711647514518
61900000 91.98693670689167
62000000 98.8369895287518
62100000 98.29195065096839
62200000 102.69325146188369
62300000 99.6403368954744
62400000 102.60833783048105
62500000 105.83139201005855
62600000 110.30493065976897
62700000 37.04718649909423
62800000 1

89100000 107.44432178950464
89200000 103.2731709940176
89300000 111.50661549217908
89400000 111.6287438616514
89500000 112.81004279188107
89600000 108.981792609283
89700000 107.8224748029466
89800000 106.51804999000872
89900000 106.18812482657354
90000000 79.33992021125093
90100000 58.90936386181553
90200000 97.84850120263803
90300000 96.69444710304342
90400000 95.48812798466855
90500000 99.23399638268656
90600000 97.78158623045087
90700000 41.47003035924683
90800000 82.13489356413079
90900000 99.79446320359284
91000000 97.39347841637749
91100000 95.54382918414228
91200000 96.68770606504583
91300000 94.9886273279234
91400000 35.055004626097165
91500000 106.09904305378228
91600000 103.44574898441351
91700000 107.39171739239087
91800000 106.06052433986112
91900000 107.15115183813195
92000000 113.15718802685332
92100000 106.08997802387495
92200000 103.4810445817639
92300000 102.57484301867079
92400000 101.66237954967887
92500000 101.1266375668742
92600000 100.84741217296326
92700000 32.07

118500000 100.0767311317517
118600000 99.2421339614018
118700000 97.01576581106268
118800000 104.61431500670714
118900000 93.17246365710663
119000000 90.31087728019186
119100000 82.87526873594754
119200000 40.64962340775233
119300000 88.44944058637496
119400000 92.86308966203002
119500000 89.98301061703684
119600000 47.15530870383011
119700000 71.42892158421098
119800000 89.89772089407685
119900000 89.92537147351163
120000000 91.5212702779169
120100000 34.54263716763475
120200000 93.07164506836043
120300000 91.01661045275635
120400000 92.02265765275989
120500000 96.20205869038847
120600000 95.37738548190083
120700000 100.67831287603758
120800000 98.01584456676912
120900000 101.87074176356238
121000000 99.89062198457816
121100000 101.60576458198923
121200000 104.19392374671409
121300000 102.03803939100182
121400000 95.87486829175762
121500000 95.630966826267
121600000 93.72596223633016
121700000 92.70802267766072
121800000 45.58229688947579
121900000 77.43861868281968
122000000 91.22783

...

982500000 1.0520308710904163
982600000 1.054492185253358
982700000 4.0195262086462895
982800000 7.503975431599599
982900000 8.317218637709864
983000000 4.027732224078916
983100000 5.135887318290093
983200000 8.254749747816485
983300000 7.686868051123476
983400000 7.154023606800581
983500000 6.785105108695289
983600000 5.629867228686661
983700000 1.91387628785571
983800000 6.619710624748809
983900000 6.269728112579557
984000000 6.665778734129226
984100000 6.945237440660632
984200000 6.0219235575787735
984300000 3.5598223632132857
984400000 3.213773952975164
984500000 6.874223224418401
984600000 6.062670360174089
984700000 6.729726129002607
984800000 6.842743923379286
984900000 5.558019411038108
985000000 2.849542795457369
985100000 4.801935277584969
985200000 6.794416287226445
985300000 6.625152564198006
985400000 6.75475187326622
985500000 6.611794325693303
985600000 4.341428599207163
985700000 2.7054946309991372
985800000 6.659485703648969
985900000 6.113585019865697
986000000 6.77284

1011000000 3.3117148452338996
1011100000 3.035860025547695
1011200000 2.4360646671117148
1011300000 0.9146346447592915
1011400000 2.970127389938978
1011500000 2.833791203448677
1011600000 3.489803163404151
1011700000 3.0555190357138233
1011800000 3.050293855460671
1011900000 1.5264174860579258
1012000000 1.7626806729530156
1012100000 2.954729763382408
1012200000 3.094030539850007
1012300000 3.2798083194730796
1012400000 2.672422189677699
1012500000 2.3776966065446494
1012600000 0.9055765288304249
1012700000 2.212178155429048
1012800000 2.7578922492324502
1012900000 2.9392290650680306
1013000000 2.69186520006858
1013100000 2.468962962866517
1013200000 1.5347830849505426
1013300000 1.1381222844979948
1013400000 2.6142566601033517
1013500000 2.4991626571539123
1013600000 2.858991729986065
1013700000 2.3613145132077746
1013800000 2.290171048996627
1013900000 0.8838826708958064
1014000000 1.759990888006313
1014100000 2.372462523465665
1014200000 2.6253028693011338
1014300000 2.5317275789361

1037000000 2.3723327768990172
1037100000 3.8228283064243933
1037200000 2.5192163702609363
1037300000 3.014788249848122
1037400000 2.912375508395828
1037500000 2.19119910723389
1037600000 2.9847151498947997
1037700000 2.364250827831442
1037800000 2.429675265023403
1037900000 2.1292087609841115
1038000000 2.8180037230806168
1038100000 2.2894679186309634
1038200000 2.7919217810028476
1038300000 2.959761842029812
1038400000 2.382748401239538
1038500000 2.6044207628258786
1038600000 2.944980024865873
1038700000 2.6544706770374993
1038800000 2.493364086539312
1038900000 3.218385358771622
1039000000 1.9937180129495202
1039100000 2.909851124734254
1039200000 2.939316479082218
1039300000 2.9166153443362277
1039400000 1.8111507387013057
1039500000 3.2448585130744014
1039600000 1.8281482717414377
1039700000 1.7084489472498126
1039800000 2.0739099838018866
1039900000 2.804849830307633
1040000000 2.106765494605731
1040100000 2.508838421919825
1040200000 2.463941171848905
best so far: 0
type: [1, 3,

1058100000 0.9972743535000617
1058200000 1.2417932755810182
1058300000 1.0872854739492357
1058400000 0.8563615540432791
1058500000 1.2181914644756415
1058600000 1.0590539429576384
1058700000 1.0012127228608676
1058800000 1.1075422531794012
1058900000 1.298229912125596
1059000000 0.9100834349762233
1059100000 1.0011728189928513
1059200000 1.1845761223399918
1059300000 0.8162067329714034
1059400000 0.9269122666919887
1059500000 1.33202918442805
1059600000 0.9597752890480676
1059700000 0.7972518961313606
1059800000 1.1449642738380366
1059900000 0.9802054859986299
1060000000 0.8859066395376186
1060100000 0.981372011572059
1060200000 1.1373353422502042
1060300000 0.7303103668232183
1060400000 0.9299353254898441
1060500000 1.1667184034672684
1060600000 0.9103854237667144
1060700000 0.7955963432152668
1060800000 1.2171208025795202
1060900000 0.8964239825269144
1061000000 0.7249697089220907
1061100000 1.059150641650084
1061200000 1.0279462001646777
1061300000 0.6861168398168762
1061400000 0.88

1084600000 0.060172904649668936
1084700000 0.05990357944301433
best so far: 0
type: [2, 1, 3, 1, 23] 138
cases of this type: 2518569
1084800000 0.144121113439803
1084900000 0.2246823316406667
1085000000 0.1996718504192432
1085100000 0.1821250665989333
1085200000 0.17965194978369672
1085300000 0.17910679758067397
1085400000 0.18393291572582326
1085500000 0.18063434023145042
1085600000 0.18193751065743302
1085700000 0.1946487637797978
1085800000 0.1114142670689106
1085900000 0.06197346485864056
1086000000 0.06121216428086493
1086100000 0.0607143930414915
1086200000 0.06027660222961704
1086300000 0.07794263529317048
1086400000 0.20137518776327967
1086500000 0.17099989613369634
1086600000 0.16733931465598342
1086700000 0.17579208581681846
1086800000 0.19410285407139924
1086900000 0.15701848938165372
1087000000 0.15351822522051334
1087100000 0.15591428690266013
1087200000 0.17089993944028284
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503
1087300000 0.14455834513059657
